# 8-9. 프로젝트 - Movielens 영화 추천 실습

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

In [1]:
# # 데이터 준비
# 1) wget으로 데이터 다운로드
# $ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

# 2) 다운받은 데이터를 작업디렉토리로 옮김
# $ mv ml-1m.zip ~/aiffel/recommendata_iu/data

# 3) 작업디렉토리로 이동
# $ cd ~/aiffel/recommendata_iu/data

# 4) 압축 해제
# $ unzip ml-1m.zip

## 1) 데이터 준비와 전처리

In [2]:
# movielens 데잍는0 rating.dat안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 정리되어 있음
import os
import pandas as pd
rating_file_path = os.getenv("HOME") + "/aiffel/recommendata_iu/data/ml-1m/ratings.dat"
ratings_cols = ["user_id", "movie_id", "rating", "timestamp"]
ratings = pd.read_csv(rating_file_path, sep="::", names=ratings_cols, engine="python")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남기기
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 칼럼 이름 count로 바꾸기
ratings.rename(columns={'rating':"count"}, inplace=True)
ratings["count"]

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
ratings.head()

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옴
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = 'ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 2) 분석해 봅시다.
- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)


In [7]:
# 유니크한 영화 수
ratings["movie_id"].nunique()

3628

In [8]:
# 유니크한 사용자 수 
ratings["user_id"].nunique()

6039

In [9]:
# 데이터 병합
data = pd.merge(movies, ratings)
data

,movie_id,title,genre,user_id,count,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5,978225952
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5,978226474
...,...,...,...,...,...,...
836473,3952,"Contender, The (2000)",Drama|Thriller,5682,3,1029457829
836474,3952,"Contender, The (2000)",Drama|Thriller,5812,4,992072099
836475,3952,"Contender, The (2000)",Drama|Thriller,5831,3,986223125
836476,3952,"Contender, The (2000)",Drama|Thriller,5837,4,1011902656


In [10]:
# 사용하는 데이터만 남김
using_cols = ["title", "user_id", "count"]
data = data[using_cols]
data.head()

,title,user_id,count
0,Toy Story (1995),1,5
1,Toy Story (1995),6,4
2,Toy Story (1995),8,4
3,Toy Story (1995),9,5
4,Toy Story (1995),10,5


In [11]:
# 첫번째 유저가 어떤 영화 봤는지 확인

condition = (data["user_id"] == data.loc[0, "user_id"])
data.loc[condition]

,title,user_id,count
0,Toy Story (1995),1,5
19773,Pocahontas (1995),1,5
35123,Apollo 13 (1995),1,5
54895,Star Wars: Episode IV - A New Hope (1977),1,4
116019,Schindler's List (1993),1,5
119008,"Secret Garden, The (1993)",1,4
130190,Aladdin (1992),1,4
138999,Snow White and the Seven Dwarfs (1937),1,4
139706,Beauty and the Beast (1991),1,5
142605,Fargo (1996),1,4


toy story (1995), toy story 2 (1999), aladdin (1992), beauty and the beast (1991), wizard of oz, the (1939)

In [12]:
# 검색을 쉽게 하기위해 영화제목 문자열을 소문자로 변경 
data["title"] = data["title"].str.lower()
data.head(10)

/home/aiffel-dj47/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,title,user_id,count
0,toy story (1995),1,5
1,toy story (1995),6,4
2,toy story (1995),8,4
3,toy story (1995),9,5
4,toy story (1995),10,5
5,toy story (1995),18,4
6,toy story (1995),19,5
7,toy story (1995),21,3
8,toy story (1995),23,4
9,toy story (1995),26,3


In [13]:
# 인기많은 영화 30개
data_count = data.groupby("title")["user_id"].count()
data_count.sort_values(ascending=False).head(10)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
Name: user_id, dtype: int64

In [14]:
# 유저별 몇명의 영화를 봤는지 통계
user_count = data.groupby("user_id")["title"].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

In [15]:
# 유저별 별점 중앙값에 대한 통계
user_median = data.groupby("user_id")["count"].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: count, dtype: float64

## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.


In [16]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ["toy story (1995)", "toy story 2 (1999)", "aladdin (1992)", "beauty and the beast (1991)", "wizard of oz, the (1939)"]

# mark라는 user_id가 위 영화를 별점을 5점씩 줬다고 가정
my_playlist  = pd.DataFrame({"user_id":["mark"]*5, "title": my_favorite, "count":[5]*5})

if not data.isin({"user_id":["mark"]})["user_id"].any(): # user_id에 mark라는 데이터가 없다면 
    data = data.append(my_playlist) # 위에 임의로 만든 my_playlist 데이터 추가
    
data.tail(10) # 추가 되었는지 확인

,title,user_id,count
836473,"contender, the (2000)",5682,3
836474,"contender, the (2000)",5812,4
836475,"contender, the (2000)",5831,3
836476,"contender, the (2000)",5837,4
836477,"contender, the (2000)",5998,4
0,toy story (1995),mark,5
1,toy story 2 (1999),mark,5
2,aladdin (1992),mark,5
3,beauty and the beast (1991),mark,5
4,"wizard of oz, the (1939)",mark,5


In [17]:
# 위에 코드에서 숫자를 ""로 묶어줘서 str으로 인식해 csr_matrix로 안 넘어갔음

## 4) CSR matrix를 직접 만들어 봅시다.


In [18]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data["user_id"].unique()
movie_unique = data["title"].unique()

In [19]:
# 유저, 아티스트 인덱싱 코드 idx는 index 약자
user_to_idx = {v:k for k,v in enumerate(user_unique)} # 유저유니크에서 k에 번호주고 v에 유저유니크 내용주고 v는 유저 k 는번호
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}
print('ok')

ok


In [20]:
print(user_unique)

[1 6 8 ... 2245 3043 'mark']


In [21]:
print(type(user_unique))

<class 'numpy.ndarray'>


In [22]:
print(movie_unique)

['toy story (1995)' 'jumanji (1995)' 'grumpier old men (1995)' ...
 'tigerland (2000)' 'two family house (2000)' 'contender, the (2000)']


In [23]:
print(type(movie_unique))

<class 'numpy.ndarray'>


In [24]:
data

,title,user_id,count
0,toy story (1995),1,5
1,toy story (1995),6,4
2,toy story (1995),8,4
3,toy story (1995),9,5
4,toy story (1995),10,5
...,...,...,...
0,toy story (1995),mark,5
1,toy story 2 (1999),mark,5
2,aladdin (1992),mark,5
3,beauty and the beast (1991),mark,5


In [25]:
# 인덱싱 확인
print(user_to_idx['mark']) # 6040명의 유저 중 마지막으로 추가된 유저이니 6039 나와야함
print(movie_to_idx["toy story (1995)"])

6039
0


In [26]:
# indexing을 통해 데이터의 컬럼내 값을 바꾸는 코드
# 딕셔너리 자료형의 get 함수는 https://wikidocs.net/16 을 참고

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 series 구하기
# 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될테니 dropna()로 제거
temp_user_data = data["user_id"].map(user_to_idx.get).dropna() # Key로 Value얻기(get) # map은 리스트의 요소를 지정된 함수로 처리해주는 함수
if len(temp_user_data) == len(data): # 모든 row가 정상적으로 인덱싱 되었다면
    print("user_id column indexing ok")
    data["user_id"] = temp_user_data # data["user_id"]를 인덱싱된 series로 교체
else: 
    print("user_id column indexing fail")
    
# artist_to_idx도 위와 동일한 방식으로 진행
temp_movie_data = data["title"].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print("title column indexing ok")
    data["title"] = temp_movie_data
else:
    print("user_id column indexing fail")
    
data

user_id column indexing ok
title column indexing ok


,title,user_id,count
0,0,0,5
1,0,1,4
2,0,2,4
3,0,3,5
4,0,4,5
...,...,...,...
0,0,6039,5
1,2845,6039,5
2,568,6039,5
3,575,6039,5


# data의 user_id, artist 컬럼 값들이 모두 정수 인덱스 값으로 변경 완료. 훈련에 필요한 데이터 전처리 완료

# 8-6. CSR(Compressed Sparse Row) Matrix

In [27]:
from scipy.sparse import csr_matrix

num_user = data["user_id"].nunique()
num_title = data["title"].nunique()

csr_data = csr_matrix((data['count'], (data.user_id, data.title)))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [28]:
print(ratings)

         user_id  movie_id  count  timestamp
0              1      1193      5  978300760
1              1       661      3  978302109
2              1       914      3  978301968
3              1      3408      4  978300275
4              1      2355      5  978824291
...          ...       ...    ...        ...
1000203     6040      1090      3  956715518
1000205     6040      1094      5  956704887
1000206     6040       562      5  956704746
1000207     6040      1096      4  956715648
1000208     6040      1097      4  956715569

[836478 rows x 4 columns]


# 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

#### als __init__ 파라미터
1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
3. use_gpu : gpu를 사용할 것인지
4. iterations : epochs과 같은 의미; 데이터를 몇번 박복해서 학습할 것인지
1, 4를 늘릴수록 학습을 잘하지만 과적합의 우려가 있음

In [29]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [79]:
# implicit alternatingleastsquares모델 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=50, dtype=np.float32)
print("ok")

ok


In [80]:
# als모델은 input으로 (item X user 꼴의 매트릭스를 받기 때문에 transpose 해줌)
csr_data_transpose = csr_data.T # T전치? 행렬변환?
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [81]:
# 모델훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/50 [00:00<?, ?it/s]

In [33]:
data.tail()

,title,user_id,count
0,0,6039,5
1,2845,6039,5
2,568,6039,5
3,575,6039,5
4,838,6039,5


In [34]:
user_to_idx

{1: 0,
 6: 1,
 8: 2,
 9: 3,
 10: 4,
 18: 5,
 19: 6,
 21: 7,
 23: 8,
 26: 9,
 28: 10,
 34: 11,
 36: 12,
 38: 13,
 44: 14,
 45: 15,
 48: 16,
 49: 17,
 51: 18,
 56: 19,
 60: 20,
 65: 21,
 68: 22,
 73: 23,
 75: 24,
 76: 25,
 78: 26,
 80: 27,
 90: 28,
 92: 29,
 96: 30,
 99: 31,
 109: 32,
 112: 33,
 114: 34,
 117: 35,
 118: 36,
 119: 37,
 121: 38,
 123: 39,
 131: 40,
 132: 41,
 134: 42,
 139: 43,
 142: 44,
 146: 45,
 147: 46,
 148: 47,
 149: 48,
 150: 49,
 152: 50,
 156: 51,
 157: 52,
 162: 53,
 163: 54,
 168: 55,
 169: 56,
 173: 57,
 175: 58,
 182: 59,
 184: 60,
 186: 61,
 187: 62,
 190: 63,
 193: 64,
 194: 65,
 195: 66,
 198: 67,
 202: 68,
 204: 69,
 213: 70,
 214: 71,
 215: 72,
 220: 73,
 223: 74,
 224: 75,
 225: 76,
 230: 77,
 231: 78,
 232: 79,
 236: 80,
 237: 81,
 239: 82,
 243: 83,
 246: 84,
 255: 85,
 258: 86,
 263: 87,
 264: 88,
 271: 89,
 272: 90,
 273: 91,
 284: 92,
 294: 93,
 299: 94,
 300: 95,
 301: 96,
 302: 97,
 306: 98,
 307: 99,
 308: 100,
 310: 101,
 314: 102,
 325: 103,
 3

In [82]:
mark, back = user_to_idx["mark"], movie_to_idx["back to the future (1985)"]
mark_vector, back_vector = als_model.user_factors[mark], als_model.item_factors[back]

print('ok')

ok


In [71]:
ratings["movie_id"]

0          1193
1           661
2           914
3          3408
4          2355
           ... 
1000203    1090
1000205    1094
1000206     562
1000207    1096
1000208    1097
Name: movie_id, Length: 836478, dtype: int64

In [83]:
mark_vector

array([-0.20402776,  0.35441157,  0.11517321, -0.11834556,  0.469708  ,
        0.27198645, -0.3632108 , -0.20985596, -0.28338996,  0.14994308,
       -0.13141477, -0.21432674, -0.32655966,  0.28788188,  0.05750332,
        0.23865278, -0.0522103 , -0.04354713,  0.19802675,  0.5884114 ,
        0.21012738,  0.18165445,  0.16394185, -0.00969   ,  0.278664  ,
        0.21727538, -0.44156912,  1.0945783 , -0.5895277 , -0.24013339,
        0.3081131 ,  0.1717963 , -0.04892254, -0.52303636, -0.16963632,
       -0.40091747,  0.1939747 , -0.06100151,  0.00355992,  0.23675996,
       -0.43130755, -0.12747768, -0.6565207 , -0.29046538, -0.46380895,
        0.08700936,  0.46830443, -0.3994231 , -0.4504686 , -0.7811763 ,
       -0.16528873, -0.456504  , -0.013855  ,  0.5375522 ,  0.16144267,
        0.25797293,  0.5044562 ,  0.4011035 ,  0.02124517,  0.49506208,
       -0.3443978 ,  0.01906   , -0.00123959, -0.36321434,  0.03091359,
       -0.33493894, -0.11371259, -0.14291479,  0.06329892, -0.19

In [84]:
back_vector

array([ 0.05894007,  0.02158402,  0.02659549,  0.02611112,  0.01210134,
        0.00980826, -0.04364069,  0.0209657 ,  0.02291127, -0.00879224,
       -0.02807744,  0.05038571,  0.06115401, -0.00580889,  0.02392456,
        0.03234718, -0.0063908 , -0.01425224, -0.01040888,  0.05360923,
       -0.01691352, -0.02163956,  0.02112262, -0.01374606,  0.01135045,
       -0.03175968, -0.00159417, -0.01118893, -0.00060638,  0.00758023,
       -0.01383123, -0.02929982, -0.00849471, -0.0077379 , -0.00231195,
        0.00837846, -0.01401641, -0.02012823,  0.02239113,  0.03227199,
        0.00029587,  0.0528164 , -0.04627914, -0.00123285,  0.02375609,
        0.02141801,  0.03314656, -0.0090758 ,  0.04047224, -0.02717731,
        0.02986938, -0.01640154,  0.00765953,  0.02358654, -0.03567661,
        0.0084885 , -0.00221786,  0.01154561,  0.02745018,  0.03266109,
        0.04493846,  0.00300044,  0.06204552, -0.01319351, -0.01935079,
        0.03005001,  0.03444051, -0.01866753,  0.0527927 ,  0.01

In [85]:
# mark와 back 내적하는 코드
np.dot(mark_vector, back_vector)

0.0029228665

In [75]:
print(ratings["movie_id"][1193])

2043


In [86]:
# 모델이 영화 Tarzan (1999) 선호도를 어떻게 예측할지 확인
tarzan = movie_to_idx["tarzan (1999)"]
tarzan_vector = als_model.item_factors[tarzan]
np.dot(mark_vector, tarzan_vector)

0.24700162

In [77]:
print(movie_to_idx["tarzan (1999)"])

2434


In [87]:
# 모델이 영화 mulan (1998) 선호도를 어떻게 예측할지 확인
mulan = movie_to_idx["mulan (1998)"]
mulan_vector = als_model.item_factors[mulan]
np.dot(mark_vector, mulan_vector)

0.24535307

# 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.
# 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.
# 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [43]:
# 내가 선호하는 5가지 중 1개
favorite_movie = 'toy story (1995)'
title = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(title, N=15)
similar_movie

[(0, 1.0000002),
 (2845, 0.82482404),
 (2114, 0.6216961),
 (33, 0.58509606),
 (568, 0.5733073),
 (1147, 0.5265186),
 (350, 0.47692236),
 (2081, 0.457543),
 (575, 0.44452482),
 (1705, 0.42387623),
 (1409, 0.38020653),
 (1689, 0.36573553),
 (2434, 0.35766506),
 (2155, 0.35478106),
 (2143, 0.34245428)]

In [44]:
# (아티스트의 id, 유사도)를 Tuple로 반환하고 있음.  
# 아티스트의 id를 다시 아티스트의 이름으로 매핑 시켜줌
# artist_to_idx 를 뒤짚어, index로부터 artist이름을 얻는 dict 생성
idx_to_movie = {v:k for k,v in movie_to_idx.items()} # items()키, 밸류 쌍으로 얻기
[idx_to_movie[i[0]] for i in similar_movie]

['toy story (1995)',
 'toy story 2 (1999)',
 "bug's life, a (1998)",
 'babe (1995)',
 'aladdin (1992)',
 'groundhog day (1993)',
 'lion king, the (1994)',
 'pleasantville (1998)',
 'beauty and the beast (1991)',
 "there's something about mary (1998)",
 'hercules (1997)',
 'mulan (1998)',
 'tarzan (1999)',
 'shakespeare in love (1998)',
 'babe: pig in the city (1998)']

In [45]:
movie_to_idx

{'toy story (1995)': 0,
 'jumanji (1995)': 1,
 'grumpier old men (1995)': 2,
 'waiting to exhale (1995)': 3,
 'father of the bride part ii (1995)': 4,
 'heat (1995)': 5,
 'sabrina (1995)': 6,
 'tom and huck (1995)': 7,
 'sudden death (1995)': 8,
 'goldeneye (1995)': 9,
 'american president, the (1995)': 10,
 'dracula: dead and loving it (1995)': 11,
 'balto (1995)': 12,
 'nixon (1995)': 13,
 'cutthroat island (1995)': 14,
 'casino (1995)': 15,
 'sense and sensibility (1995)': 16,
 'four rooms (1995)': 17,
 'ace ventura: when nature calls (1995)': 18,
 'money train (1995)': 19,
 'get shorty (1995)': 20,
 'copycat (1995)': 21,
 'assassins (1995)': 22,
 'powder (1995)': 23,
 'leaving las vegas (1995)': 24,
 'othello (1995)': 25,
 'now and then (1995)': 26,
 'persuasion (1995)': 27,
 'city of lost children, the (1995)': 28,
 'shanghai triad (yao a yao yao dao waipo qiao) (1995)': 29,
 'dangerous minds (1995)': 30,
 'twelve monkeys (1995)': 31,
 'wings of courage (1995)': 32,
 'babe (1995)'

In [46]:
# 위 코드 함수화
def get_similar_movie(movie_name: str):
    title = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(title)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

print("ok")

ok


In [47]:
# 내가 고르지 않은 영화 중 1개
get_similar_movie('american beauty (1999)')

['american beauty (1999)',
 'being john malkovich (1999)',
 'election (1999)',
 'shakespeare in love (1998)',
 'fargo (1996)',
 'braveheart (1995)',
 'saving private ryan (1998)',
 'american pie (1999)',
 'pulp fiction (1994)',
 'sixth sense, the (1999)']

In [48]:
user = user_to_idx['mark']
# recommend에서 user*item CSR Matrix를 받음
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(2114, 0.59058934),
 (350, 0.55254364),
 (33, 0.47041112),
 (1861, 0.36681712),
 (532, 0.27602124),
 (941, 0.2736119),
 (1689, 0.25988892),
 (2434, 0.25849962),
 (574, 0.25243855),
 (1164, 0.24539459),
 (1409, 0.23472817),
 (2155, 0.23196179),
 (1858, 0.21966974),
 (2056, 0.20057419),
 (1860, 0.19623187),
 (3433, 0.19424018),
 (818, 0.19190134),
 (2506, 0.18875208),
 (1003, 0.18643793),
 (725, 0.18474479)]

In [49]:
[idx_to_movie[i[0]] for i in movie_recommended]

["bug's life, a (1998)",
 'lion king, the (1994)',
 'babe (1995)',
 'little mermaid, the (1989)',
 'nightmare before christmas, the (1993)',
 'mary poppins (1964)',
 'mulan (1998)',
 'tarzan (1999)',
 'snow white and the seven dwarfs (1937)',
 'fantasia (1940)',
 'hercules (1997)',
 'shakespeare in love (1998)',
 'jungle book, the (1967)',
 'antz (1998)',
 'lady and the tramp (1955)',
 'chicken run (2000)',
 "singin' in the rain (1952)",
 'iron giant, the (1999)',
 'e.t. the extra-terrestrial (1982)',
 'hunchback of notre dame, the (1996)']

# 총평

1. csr matrix가 정상적으로 만들어졌다.
- 처음에는 정상적으로 만들지 못했다. 데이터병합을 최종으로 할 생각을 하고 movie_id기준으로 작업을 했는데 뭐가 잘못됐는지 잘 안됐다. 이렇게 저렇게 바꿔서 결국 csr matrix까지는 만들었지만 그 후 모델에 적용이 안됐다.
- 그래서 movies와 ratings를 초반에 병합했고 병합을 해보니 나중에 데이터를 보기도 편해졌고 이후에는 큰 문제 없이 잘 됐다.
    
2. mf모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.
- 나는 만화 종류의 영화를 좋아하는 영화로 둔 후 백 투더 퓨처와의 내적을 해보니 약 0.4의 값이 나왔다. 추가로 tarzan과 mulan을 해보니 약 0.2의 더 낮은 값이 나왔다.. 내가 고른 영화는 만화영화여서 tarzan과 mulan에 더 큰 값이 나와야하는데 왜 그런지 잘 모르겠었다.
- 그래서 모델의 파라미터를 수정해서 해보니 back to the future는 0.003, tarzan과 mulan은 0.2로 조금 더 내가 원하는 방향으로 나왔다. 

3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다. 
- toy story를 넣으니 toy story 시리즈를 추천해줬고 알라딘, 라이언킹 등 만화 종류를 많이 추천해줬다.
- user가 제일 많이 본 영화가 american beauty이길래 그걸 넣어본 후 추천영화와 american beauty에 대해 검색해보니 추천 영화들의 설명란에 '욕망'이란 단어가 꼭 들어간다. 비슷하게 추천해준 것 같다.

4. 느낀점
- 익스를 할 때마다 느끼는 것이지만 코드를 조금 더 자세히, 섬세하게, 조금만 다르게 생각해보고 정해진 틀에서 좀 벗어나려고 노력해야겠다고 매번 느낀다. 분명 해결할 수 있는 것인데 뭔가 답답한 느낌이다.